<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/stefan-bozman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Stefan–Boltzmann Interactive Calculator + Dynamic Plot (fixed array handling)
# Works in Google Colab

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Stefan–Boltzmann constant [W m^-2 K^-4]
sigma = 5.670374419e-8

# --- Core conversions ---
def flux_from_temperature(T_K):
    """Return flux F [W/m^2] from temperature T [K]."""
    T_K = np.maximum(T_K, 0.0)   # element-wise max, works for scalars and arrays
    return sigma * T_K**4

def temperature_from_flux(F):
    """Return temperature T [K] from flux F [W/m^2]."""
    F = np.maximum(F, 0.0)
    return np.where(F > 0, (F / sigma) ** 0.25, 0.0)

def to_kelvin(T_value, unit):
    """Convert input temperature to Kelvin."""
    if unit == "°C":
        return np.maximum(T_value + 273.15, 0.0)
    return np.maximum(T_value, 0.0)

def from_kelvin(T_K):
    """Return (K, °C)."""
    return T_K, T_K - 273.15

# Reference curve domain (log-spaced to span many orders of magnitude)
T_min, T_max = 1.0, 1.5e7  # K
T_curve = np.logspace(np.log10(T_min), np.log10(T_max), 600)
F_curve = flux_from_temperature(T_curve)

# Reference markers
refs = {
    "Earth eff. ~255 K": 255.0,
    "Room ~288 K": 288.0,
    "Sun surf. ~5778 K": 5778.0,
    "Solar core ~1.5e7 K": 1.5e7
}

# Widgets
mode_dd   = widgets.Dropdown(options=["Temperature → Flux", "Flux → Temperature"],
                             value="Temperature → Flux", description="Mode")
unit_dd   = widgets.Dropdown(options=["K", "°C"], value="K", description="T unit")
T_slider  = widgets.FloatLogSlider(value=300.0, base=10, min=0, max=7.1761, step=0.001,
                                   description="T", readout_format=".3g")  # ~1 K .. 1.5e7 K
F_slider  = widgets.FloatLogSlider(value=1361.0, base=10, min=0, max=12, step=0.001,
                                   description="F (W/m²)", readout_format=".3g")
# F slider spans 1 .. 1e12 W/m²

def ui(mode, unit, T, F):
    plt.figure(figsize=(7,5))

    if mode == "Temperature → Flux":
        T_K = to_kelvin(T, unit)
        F_sel = flux_from_temperature(T_K)
        T_K_out, T_C_out = from_kelvin(T_K)

        print(f"T = {T_K_out:.3g} K  ({T_C_out:.3g} °C)")
        print(f"Flux F = {F_sel:.3e} W/m²")

        plt.loglog(T_curve, F_curve)
        plt.scatter([T_K], [F_sel], s=40, color="red")

    else:
        F_sel = F
        T_K = temperature_from_flux(F_sel)
        T_K_out, T_C_out = from_kelvin(T_K)

        print(f"F = {F_sel:.3e} W/m²")
        print(f"T = {T_K_out:.3g} K  ({T_C_out:.3g} °C)")

        plt.loglog(T_curve, F_curve)
        if T_K > 0:
            plt.scatter([T_K], [F_sel], s=40, color="blue")

    # Reference markers
    for label, T_ref in refs.items():
        F_ref = flux_from_temperature(T_ref)
        plt.scatter([T_ref], [F_ref], s=15)
        plt.text(T_ref, F_ref, f"  {label}", fontsize=9)

    plt.xlabel("Temperature T (K)")
    plt.ylabel("Radiative flux F (W/m²)")
    plt.title("Stefan–Boltzmann: F = σ T^4")
    plt.grid(True, which="both", linestyle=":")
    plt.show()

# Interactive UI
interact(ui, mode=mode_dd, unit=unit_dd, T=T_slider, F=F_slider)

interactive(children=(Dropdown(description='Mode', options=('Temperature → Flux', 'Flux → Temperature'), value…

<function __main__.ui(mode, unit, T, F)>